# Spherical harmonic expansion of the Doppler operator

Given a rigidly star rotating viewed at an inclination of $90^\circ$ with zero obliquity, the radial component of the velocity field is just $v(x, y) = x$. When Taylor expanding the Doppler operator, we need to know what integer powers of this expression look like when expressed in terms of spherical harmmonics. Below we compute the corresponding spherical harmonic coefficients.

## Imports

First, let's import some stuff. We'll use `sympy` for the derivations.

In [1]:
import matplotlib.pyplot as pl
%matplotlib notebook
import numpy as np
from IPython.display import display, Math
import sympy
from sympy import *
from sympy.functions.special.tensor_functions import KroneckerDelta
print("Using sympy version", sympy.__version__)

# Initialize the session
init_session(quiet=True)

# Define our symbols
x, y, z, mu, nu, l, m, j, k, p, q, n, A, B, C, alpha, omeq = symbols('x y z mu nu l m j k p q n A B C alpha \omega_{eq}')

Using sympy version 1.3



## Polynomials to spherical harmonics

As in Luger et al. (2018), let's compute the change of basis matrix from polynomials to spherical harmonic coefficients. This is the inverse of the $A_1$ matrix introduced in Luger et al. (2018). Note that it includes the normalization of $\frac{2}{\sqrt{\pi}}$ used internally by `starry`.

In [6]:
def poly_basis(n, x, y):
    """Return the n^th term in the polynomial basis."""
    l = Rational(floor(sqrt(n)))
    m = Rational(n - l * l - l)
    mu = Rational(l - m)
    nu = Rational(l + m)
    if (nu % 2 == 0):
        i = Rational(mu, 2)
        j = Rational(nu, 2)
        k = Rational(0)
    else:
        i = Rational(mu - 1, 2)
        j = Rational(nu - 1, 2)
        k = Rational(1)
    return x ** i * y ** j * sqrt(1 - x ** 2 - y ** 2) ** k

def Coefficient(expression, term):
    """Return the coefficient multiplying `term` in `expression`."""
    # Get the coefficient
    coeff = expression.coeff(term)
    # Set any non-constants in this coefficient to zero. If the coefficient
    # is not a constant, this is not the term we are interested in!
    coeff = coeff.subs(sqrt(1 - x ** 2 - y ** 2), 0).subs(x, 0).subs(y, 0)
    return coeff

def SA(l, m):
    """A spherical harmonic normalization constant."""
    return sqrt((2 - KroneckerDelta(m, 0)) * (2 * l + 1) * factorial(l - m) / (4 * pi * factorial(l + m)))

def SB(l, m, j, k):
    """Another spherical harmonic normalization constant."""
    try: 
        ratio = factorial(Rational(l + m + k - 1, 2)) / factorial(Rational(-l + m + k - 1,  2))
    except ValueError:
        ratio = 0
    res = 2 ** l * Rational(factorial(m), (factorial(j) * factorial(k) * factorial(m - j) * factorial(l - m - k))) * ratio
    return simplify(res)

def SC(p, q, k):
    """Return the binomial theorem coefficient `C`."""
    res = factorial(Rational(k, 2)) / (factorial(Rational(q, 2)) * factorial(Rational(k - p, 2)) * factorial(Rational(p - q, 2)))
    return simplify(res)

def Y(l, m, x, y):
    """Return the spherical harmonic of degree `l` and order `m`."""
    res = 0
    z = sqrt(1 - x ** 2 - y ** 2)
    if (m >= 0):
        for j in range(0, m + 1, 2):
            for k in range(0, l - m + 1, 2):
                for p in range(0, k + 1, 2):
                    for q in range(0, p + 1, 2):
                        res += (-1) ** ((j + p) // 2) * SA(l, m) * SB(l, m, j, k) * SC(p, q, k) * x ** (m - j + p - q) * y ** (j + q)
            for k in range(1, l - m + 1, 2):
                for p in range(0, k, 2):
                    for q in range(0, p + 1, 2):
                        res += (-1) ** ((j + p) // 2) * SA(l, m) * SB(l, m, j, k) * SC(p, q, k - 1) * x ** (m - j + p - q) * y ** (j + q) * z          
    else:
        for j in range(1, abs(m) + 1, 2):
            for k in range(0, l - abs(m) + 1, 2):
                for p in range(0, k + 1, 2):
                    for q in range(0, p + 1, 2):
                        res += (-1) ** ((j + p - 1) // 2) * SA(l, abs(m)) * SB(l, abs(m), j, k) * SC(p, q, k) * x ** (abs(m) - j + p - q) * y ** (j + q)
            for k in range(1, l - abs(m) + 1, 2):
                for p in range(0, k, 2):
                    for q in range(0, p + 1, 2):
                        res += (-1) ** ((j + p - 1) // 2) * SA(l, abs(m)) * SB(l, abs(m), j, k) * SC(p, q, k - 1) * x ** (abs(m) - j + p - q) * y ** (j + q) * z

    return res

def p_Y(l, m, lmax):
    """Return the polynomial basis representation of the spherical harmonic `Y_{lm}`."""
    ylm = Y(l, m, x, y)
    res = [ylm.subs(sqrt(1 - x ** 2 - y ** 2), 0).subs(x, 0).subs(y, 0)]
    for n in range(1, (lmax + 1) ** 2):
        res.append(Coefficient(ylm, poly_basis(n, x, y)))
    return res

def A1(lmax, norm = 2 / sqrt(pi)):
    """Return the change of basis matrix A1. The columns of this matrix are given by `p_Y`."""
    res = zeros((lmax + 1) ** 2, (lmax + 1) ** 2)
    n = 0
    for l in range(lmax + 1):
        for m in range(-l, l + 1):
            res[n] = p_Y(l, m, lmax)
            n += 1
    return res * norm

We can now evaluate the change of basis matrix from spherical harmonic coefficients to polynomials, $A_1$ for $l_\mathrm{max} = 2$. We then take the inverse to go from polynomial coeffiecients to $Y_{lm}$ coefficients:

In [7]:
M = Matrix(A1(2)).inv()
M

⎡                       π                             π   ⎤
⎢π   0     0     0      ─       0      0      0       ─   ⎥
⎢                       3                             3   ⎥
⎢                                                         ⎥
⎢               √3⋅π                                      ⎥
⎢0   0     0    ────    0       0      0      0       0   ⎥
⎢                3                                        ⎥
⎢                                                         ⎥
⎢         √3⋅π                                            ⎥
⎢0   0    ────   0      0       0      0      0       0   ⎥
⎢          3                                              ⎥
⎢                                                         ⎥
⎢   √3⋅π                                                  ⎥
⎢0  ────   0     0      0       0      0      0       0   ⎥
⎢    3                                                    ⎥
⎢                                                         ⎥
⎢                                    √15

## $1$

In [14]:
vec = Matrix([1, 0, 0, 0, 0, 0, 0, 0, 0])
ycoeffs = simplify(M * vec)
ycoeffs.T

[π  0  0  0  0  0  0  0  0]

## $x$

In [12]:
vec = Matrix([0, 1, 0, 0, 0, 0, 0, 0, 0])
ycoeffs = simplify(M * vec)
ycoeffs.T

⎡         √3⋅π               ⎤
⎢0  0  0  ────  0  0  0  0  0⎥
⎣          3                 ⎦

## $x^2$

In [13]:
vec = Matrix([0, 0, 0, 0, 1, 0, 0, 0, 0])
ycoeffs = simplify(M * vec)
ycoeffs.T

⎡π                 -√5⋅π      √15⋅π⎤
⎢─  0  0  0  0  0  ──────  0  ─────⎥
⎣3                   15         15 ⎦